# Regression: Multiple Regression (gradient descent)

The goal of the code below is to generate a simple regression model (that takes a single input to make a prediction) and compare its performance to a multiple regression model (that takes more than one input to make a prediction.) The models are built in the context of the input, gene expression of a gene familiy, and how it affects the output, effector gene expression. The goal is test this code with existing gene expression repositories such as the NIH Gene Expression Omni

In [ ]:
import graphlab

In [ ]:
gene_data = graphlab.SFrame('gene_expression_panel.gl/')

# Convert to Numpy Array

In [ ]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [ ]:
# below is a user defined function used to quickly convert data sets into a "numpy" matrix
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 #add a column for a constant to generate a y-intercept
    features = ['constant'] + features 
    features_sframe = data_sframe[features] #extract the features/genes of interest in addition to the column with the constants
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [ ]:
#input the column headers that contain the feature of interest such as 'gene_family' and the output of interest 'effector_gene_expression'
#from the collection of data named 'gene_data'
(example_features, example_output) = get_numpy_data(gene_data, ['gene_family1'], 'effector_gene_expression') # the [] around 'sqft_living' makes it a list
#convert the data type into a matrix for downstream calculations

# Predicting output given regression weights

In [ ]:
def predict_output(feature_matrix, weights):
    #once we have the initial weights of each feature--gene expression of a gene family--we can predict the expression levels of the 
    #effector gene by the for an individual sample.
    #KEY: this is for an individual sample and we will ultimately iterate through many rows(samples) of a feature/gene expression
    #matrix. Afterwards we will optimize the model by finding the version of the model with the
    #smallest residual sum of squares value.
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Computing the Derivative of the Squared Difference of Predicted Values
# and Actual Outputs to Calculate the Weights of Features/inputs of Gene Expression

In [ ]:
#we need to find the derivative of each feature over all data points and the errors(difference between predictions and
#actual outputs according to data. This is necessary to determine the weights of each feature that will be in the final
#model
def feature_derivative(errors, feature):
    dot_prod = np.dot(errors, feature)
    derivative = 2*dot_prod #simplified and cleaned up derivative of the squared difference of predicted outputs and actual
    #outputs of the data
    return(derivative)

# Gradient Descent (used to find the regression model with the minimized residual sum squared error)

In [ ]:
from math import sqrt

In [ ]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights)
    while not converged:
        predictions = predict_output(feature_matrix, weights)
        errors = predictions - output
        gradient_sum_squares = 0
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            derivative = feature_derivative(errors, feature_matrix[:, i])
            gradient_sum_squares = derivative*derivative + gradient_sum_squares
            weights[i] = weights[i] - (derivative*step_size)
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

# Running the Gradient Descent as Simple Regression and Putting all of the User defined Functions Above Together

In [ ]:
train_data,test_data = sales.random_split(.8,seed=0)
#split the data into a training set to find the weights of features and to find the optimal model from this subset of
#data.

#the test data is used to predict the expected output given a feature/gene expression level the model has not "seen" before
#afterwards the error of this model will be calculated according to the actual outputs that the model has not 'seen' before
#as well.

In [ ]:

simple_features = ['gene_family1']
my_output = 'effector_gene'


(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output) ###split the training data
#into the features/input gene expression and the actual output/effector gene expression. Remember, the actual output is
#used to check how accurate the model predictions are.

initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [ ]:
model1_weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)
#the regression_gradient_function gives us the weights of the features that will be used to make the predicted
#outputs

In [ ]:
#the test data is split into the feature inputs and outputs just like the training data. The goal is to feed the model
#inputs/gene expression levels it has not "seen" before and then we use the test data outputs that correspond to the inputs
#to see how well the model performs

(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

In [ ]:
model1_predictions = predict_output(test_simple_feature_matrix, model1_weights)

In [ ]:
#to see how well the model did, the residual sum squared (RSS) needs to be calculated
#afterwards another model is generated but with more input features to be trained on
#then on this second model we will predict how well the second model performs on test data again
#lastly, we will compare the RSS of these two models to see which is the best of the two.
model1_residual_square = (model1_predictions - test_output)**2
model1_RSS = model1_residual_square.sum()
print model1_RSS

# Running a multiple regression

In [ ]:
model_features = ['gene_family1', 'gene_family2'] # this time we will use two gene families with the intention that the
#model will perform better with more information

my_output = 'effector gene'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
(test_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [ ]:
model2_weights = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)

In [ ]:

model2_predictions = predict_output(test_feature_matrix, model2_weights)

In [ ]:
model2_residual_square = (model2_predictions - test_output)**2
model2_RSS = model2_residual_square.sum()
print model2_RSS